In [1]:
import cmdprod as cp
import os
import kbrgan.glo as glo

In [9]:
import kbrgan.ex.exutil as exu
import random

list_files = exu.list_files_of_extensions(glo.share_path('sample_list/celeba_list'),['.txt'])


In [11]:
len(list_files)

15

In [43]:
def args_spec(logdir, list_extractor_type=['vgg_face'],
              list_extractor_layers=['8 17 26 35'],):
    '''
    Return a specification arguments (including candidate values) for 
    an experiment with Lars's models.
    
    #!/bin/bash

    python3 run_lars_gkmm.py \
        --extractor_type vgg_face \
        --extractor_layers 8 17 26 35 \
        --texture 1\
        --depth_process no \
        --g_path gan_data/celebAHQ_00/chkpts/model.pt \
        --g_type celebAHQ.yaml \
        --g_min -1.0 \
        --g_max 1.0 \
        --logdir log_lars_celeba_vggface/ \
        --device gpu \
        --n_sample 1 \
        --n_opt_iter 3000 \
        --lr 1e-1 \
        --seed 9 \
        --img_log_steps 10 \
        --cond_path  c.txt\
        --kernel imq \
        --kparams -0.5 1e+2 \
    '''
    # constants
    p_texture = cp.Param('texture', [0])
    p_depth_process = cp.Param('depth_process', ['no'])
    
    place_extractor = 'resnet50_365'
    #p_g_path = cp.Param('g_path', [g_path])
    p_g_type = cp.ParamGroup(
        ['g_type', 'g_path','cond_path','extractor_type','extractor_layer'],
        [
            ('celebAHQ.yaml', 'gan_data/celebAHQ_00/chkpts/model.pt',
             [exu.list_files_of_extensions(glo.share_path('sample_list/celeba_list'),['.txt'])],
             ['vgg_face'] ,[['17'], ['26'], ['35']]
                                          
             ),
            
            ('cub.yaml', 'CUB_data/model.pt', 
             [exu.list_files_of_extensions(glo.share_path('sample_list/cub_list'),['.txt'])],
            ['vgg'] ,[['17'], ['26'], ['35']]
             
            ),
            
            ('lsun_bridge.yaml', 'gan_data/lsun_bridge/chkpts/model.pt', 
             [exu.list_files_of_extensions(glo.share_path('sample_list/bridge_list'),['.txt'])],
            [place_extractor], []
            ),
            
            ('lsun_bedroom.yaml', 'gan_data/lsun_bedrooms/chkpts/model.pt', 
             [exu.list_files_of_extensions(glo.share_path('sample_list/bedroom_list'),['.txt'])],
            [place_extractor], []
            ),

            ('lsun_tower.yaml', 'gan_data/lsun_tower/chkpts/model.pt', 
             [exu.list_files_of_extensions(glo.share_path('sample_list/tower_list'),['.txt'])],
            [place_extractor], []
            ),
            
            ('mnist_dcgan', 'prob_models/mnist_dcgan/mnist_dcgan_ep40_bs64.pt', 
             [exu.list_files_of_extensions(glo.share_path('sample_list/tower_list'),['.txt'])],
            [place_extractor], []
            )

        ],
    )
       
    p_g_min = cp.Param('g_min', [-1.0])
    p_g_max = cp.Param('g_max', [1.0])
    p_logdir = cp.Param('logdir', [logdir])
    p_device = cp.Param('device', ['gpu'])
    p_n_opt_iter = cp.Param('n_opt_iter', [3000])
    p_img_log_steps = cp.Param('img_log_steps', [1000])

    # with some candidate values to try
    p_n_sample = cp.Param('n_sample', [1]) 
    p_lr = cp.Param('lr', [5e-2,5e-3])
    p_seed = cp.Param('seed', [9])
    #p_cond_path = cp.Param('cond_path', list_cond_path)

    p_kgroup = cp.ParamGroup(
        ['kernel', 'kparams'],
        [
            ('linear', []),
            # ('gauss', 10.0), 
            #('imq', [-0.5, 1.0]),
            ('imq', [-0.5, 100.0])
        ],
        
    )
    args = cp.Args([p_texture,
                    p_depth_process, p_g_type, p_g_min, p_g_max,
                    p_logdir, p_device, p_n_opt_iter, p_img_log_steps,
                    p_n_sample, p_lr, p_seed, p_kgroup])
    return args

In [44]:


bundle1 = dict(
    logdir='log/',
)
args = args_spec(**bundle1)

line_begin = 'python3 ../run_lars_gkmm.py '
# potentially the destination script folder name should depend on the type
# of bundle
dest_path = 'sample_list_all_run'
token_path = 'tokens'
# print to stdout
# args_processor = cp.APPrint(prefix=line_begin, suffix='\n')
# args_processor.iaf.value_formatter.list_value_sep = ', '
# args_processor(args)

# generate Bash files in a folder
iaf = cp.IAFArgparse(
    # separtor for different pair-value pairs
    pv_sep=' \\\n'
)
args_file_proc = cp.APPerBashFile(
    dest_path=dest_path,
    token_path=token_path,
    iaf=iaf,
    file_begin='#!/bin/bash',
    file_end='# end of script',
    line_begin=line_begin,
    line_end='',
)
# for values which are lists, separate items in the list with the following
# string
args_file_proc.iaf.value_formatter.list_value_sep = ' '
args_file_proc(args)

In [36]:
args